In [1]:
import xml.etree.ElementTree as ET


def values(elements, node, tag):
    return [element[node].get(tag) for element in elements]

In [2]:
class Vector3(object):
    def __init__(self, x: float, y: float, z: float):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __repr__(self):  # for debugging
        return f"Vector3({self.x}, {self.y}, {self.z})"

    def __str__(self):  # for printing
        return f"{self.x}, {self.y}, {self.z}"

    def __eq__(self, other):
        """Overload equality operator."""
        if not isinstance(other, Vector3):
            return False
        return (self.x == other.x) and (self.y == other.y) and (self.z == other.z)

    def __add__(self, other):
        """Overload addition operator."""
        if isinstance(other, Vector3):
            x = self.x + other.x
            y = self.y + other.y
            z = self.z + other.z
        else:
            x = self.x + other
            y = self.y + other
            z = self.z + other
        return Vector3(x, y, z)

    def __sub__(self, other):
        """Overload subtraction operator."""
        if isinstance(other, Vector3):
            x = self.x - other.x
            y = self.y - other.y
            z = self.z - other.z
        else:
            x = self.x - other
            y = self.y - other
            z = self.z - other
        return Vector3(x, y, z)

    def __mul__(self, other):
        """Overload multiplication operator."""
        if isinstance(other, Vector3):
            x = self.x * other.x
            y = self.y * other.y
            z = self.z * other.z
        else:
            x = self.x * other
            y = self.y * other
            z = self.z * other
        return Vector3(x, y, z)

    def __truediv__(self, other):
        """Overload division operator."""
        if isinstance(other, Vector3):
            x = self.x / other.x
            y = self.y / other.y
            z = self.z / other.z
        else:
            x = self.x / other
            y = self.y / other
            z = self.z / other
        return Vector3(x, y, z)

    def magnitude(self):
        """Compute magnitude (length)."""
        x = self.x**2
        y = self.y**2
        z = self.z**2
        return sqrt(x + y + z)
    

class Quaternion(object):
    def __init__(self, w: float, x: float, y: float, z: float):
        self.w = float(w)
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def __repr__(self):  # for debugging
        return f"Quaternion({self.w}, {self.x}, {self.y}, {self.z})"

    def __str__(self):  # for printing
        return f"{self.w}, {self.x}, {self.y}, {self.z}"

    def __eq__(self, other):
        """Overload equality operator."""
        if not isinstance(other, Quaternion):
            return False
        return (self.w == other.w) and (self.x == other.x) and (self.y == other.y) and (self.z == other.z)

    def __add__(self, other):
        """Overload addition operator."""
        if isinstance(other, Quaternion):
            w = self.w + other.w
            x = self.x + other.x
            y = self.y + other.y
            z = self.z + other.z
        else:
            w = self.w + other
            x = self.x + other
            y = self.y + other
            z = self.z + other
        return Quaternion(w, x, y, z)

    def __sub__(self, other):
        """Overload subtraction operator."""
        if isinstance(other, Quaternion):
            w = self.w - other.w
            x = self.x - other.x
            y = self.y - other.y
            z = self.z - other.z
        else:
            w = self.w - other
            x = self.x - other
            y = self.y - other
            z = self.z - other
        return Quaternion(w, x, y, z)

    def __mul__(self, other):
        """Overload multiplication operator."""
        if isinstance(other, Quaternion):
            w = self.w * other.w
            x = self.x * other.x
            y = self.y * other.y
            z = self.z * other.z
        else:
            w = self.w * other
            x = self.x * other
            y = self.y * other
            z = self.z * other
        return Quaternion(w, x, y, z)

    def __truediv__(self, other):
        """Overload division operator."""
        if isinstance(other, Quaternion):
            w = self.w / other.w
            x = self.x / other.x
            y = self.y / other.y
            z = self.z / other.z
        else:
            w = self.w / other
            x = self.x / other
            y = self.y / other
            z = self.z / other
        return Quaternion(w, x, y, z)

    def magnitude(self):
        """Compute magnitude (length)."""
        w = self.w**2
        x = self.x**2
        y = self.y**2
        z = self.z**2
        return sqrt(w + x + y + z)

# 1. Dataset construction

In [3]:
import csv

objects = ["Argos", "Orion", "Teuthus", "Vermis"]

## 1.1. Weapon (MobileObject | Cannon)

In [4]:
class MobileObject(object):
    def __init__(self, mobility_type: int, local_position: Vector3):
        self.mobility_type  = int(mobility_type)
        self.local_position = local_position
    
    def __repr__(self):
        return f"MobileObject(mobility_type: {self.mobility_type}, local_position: <{self.local_position}>)"


class Cannon(object):
    def __init__(self, firing_turn_maximum: int, firing_turn_start: int, firing_turn_end: int,
                       roll: float, firing_position: Vector3, firing_direction: Vector3):
        self.firing_turn_maximum = int(firing_turn_maximum)
        self.firing_turn_start   = int(firing_turn_start)
        self.firing_turn_end     = int(firing_turn_end)
        self.roll                = float(roll)
        self.firing_position     = firing_position
        self.firing_direction    = firing_direction
    
    def __repr__(self):
        return f"Cannon(firing_turn_maximum: {self.firing_turn_maximum}, firing_turn_start: {self.firing_turn_start}, firing_turn_end: {self.firing_turn_end}, roll: {self.roll}, firing_position: <{self.firing_position}>, firing_direction: <{self.firing_direction}>)"

In [5]:
class WeaponWithMobileObjects(object):
    def __init__(self, scale: Vector3, position: Vector3, orientation: Quaternion,
                       hull_index_parent: int, local_direction: Vector3,
                       mobile_objects: list[MobileObject]):
        self.scale              = scale
        self.position           = position
        self.orientation        = orientation
        self.hull_index_parent  = int(hull_index_parent)
        self.local_direction    = local_direction
        self.mobile_objects     = mobile_objects


class WeaponWithCannons(object):
    def __init__(self, scale: Vector3, position: Vector3, orientation: Quaternion,
                       hull_index_parent: int, local_direction: Vector3,
                       cannons: list[Cannon]):
        self.scale              = scale
        self.position           = position
        self.orientation        = orientation
        self.hull_index_parent  = int(hull_index_parent)
        self.local_direction    = local_direction
        self.cannons            = cannons

In [6]:
nodes = {"ObjectData": 0, "ComponentData": 3, "WeaponCannonData": 0}
datasets = {}

for o in objects:
    xml = ET.parse(f"{o}.xml").getroot()
    
    print(f"###### Working with {o}.xml #######")
    
    ### MobileObjects ###
    mobile_objects = xml.findall("WEAPONRYWEAPONSAI/Weapon/MobileObjects")
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon/MobileObjects': %d\n" % len(mobile_objects))
    
    if len(mobile_objects) > 0:
        mobile_objects_number = list(map(int, [mobile_object.get("Number") for mobile_object in mobile_objects]))
        print(f"MobileObjectsNumber: {mobile_objects_number}\n")

        mobility_types = list(map(int, [mobile_object.get("MobilityType") for mobile_object in mobile_objects]))
        mobility_types_ = [mobility_types[no] for no, rep in enumerate(mobile_objects_number) for reps in range(rep)]
        print(f"MobilityTypes: {mobility_types_}\n")

        local_positions = [Vector3(node.attrib.get("LocalPositionX"), node.attrib.get("LocalPositionY"), node.attrib.get("LocalPositionZ")) for node in xml.findall("WEAPONRYWEAPONSAI/Weapon/MobileObjects/MobileObject")]
        local_positions_ = [local_positions[no] for no, rep in enumerate(mobile_objects_number) for reps in range(rep)]
        print(f"LocalPositions: {local_positions}\n")
        
        mobile_objects_ = [MobileObject(mobility_types_[i], local_positions_[i]) for i in range(len(xml.findall("WEAPONRYWEAPONSAI/Weapon/MobileObjects/MobileObject")))]
        print(f"{mobile_objects_}\n")
    
    ### WeaponsWithMobileObjects ###
    weapons = xml.findall("WEAPONRYWEAPONSAI/Weapon")
    weapons[:] = [weapon for weapon in weapons if weapon.find("MobileObjects") is not None]
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon': %d\n" % len(weapons))
    
    if len(weapons) > 0:
        scales_x = values(weapons, nodes["ObjectData"], "ScaleX")
        scales_y = values(weapons, nodes["ObjectData"], "ScaleY")
        scales_z = values(weapons, nodes["ObjectData"], "ScaleZ")
        scales = [Vector3(scales_x[i], scales_y[i], scales_z[i]) for i in range(len(weapons))]
        print(f"Scales: {scales}\n")

        positions_x = values(weapons, nodes["ObjectData"], "PositionX")
        positions_y = values(weapons, nodes["ObjectData"], "PositionY")
        positions_z = values(weapons, nodes["ObjectData"], "PositionZ")
        positions = [Vector3(positions_x[i], positions_y[i], positions_z[i]) for i in range(len(weapons))]
        print(f"Positions: {positions}\n")

        orientations_w = values(weapons, nodes["ObjectData"], "OrientationW")
        orientations_x = values(weapons, nodes["ObjectData"], "OrientationX")
        orientations_y = values(weapons, nodes["ObjectData"], "OrientationY")
        orientations_z = values(weapons, nodes["ObjectData"], "OrientationZ")
        orientations = [Quaternion(orientations_w[i], orientations_x[i], orientations_y[i], orientations_z[i]) for i in range(len(weapons))]
        print(f"Orientations: {orientations}\n")

        hull_index_parents = list(map(int, values(weapons, nodes["ComponentData"], "HullIndexParent")))
        print(f"HullIndexParents: {hull_index_parents}\n")

        local_directions_x = values(weapons, nodes["ComponentData"], "LocalDirectionX")
        local_directions_y = values(weapons, nodes["ComponentData"], "LocalDirectionY")
        local_directions_z = values(weapons, nodes["ComponentData"], "LocalDirectionZ")
        local_directions = [Vector3(local_directions_x[i], local_directions_y[i], local_directions_z[i]) for i in range(len(weapons))]
        print(f"LocalDirections: {local_directions}\n")

        k, weapons_with_mobile_objects = 0, []
        for i in range(len(mobile_objects_number)):
            mobile_objects__ = []
            for j in range(mobile_objects_number[i]):
                mobile_objects__ += [mobile_objects_[k]]
                k += 1
            weapons_with_mobile_objects += [WeaponWithMobileObjects(scales[i], positions[i], orientations[i], hull_index_parents[i], local_directions[i], mobile_objects__)]

        dataset = []
        for i in range(len(weapons_with_mobile_objects)):
            for j in range(len(weapons_with_mobile_objects[i].mobile_objects)):
                    dataset.append({"Name": f"{o}-baseline",
                                    "HullIndexParent": weapons_with_mobile_objects[i].hull_index_parent,
                                    "ScaleX": weapons_with_mobile_objects[i].scale.x,
                                    "ScaleY": weapons_with_mobile_objects[i].scale.y,
                                    "ScaleZ": weapons_with_mobile_objects[i].scale.z,
                                    "PositionX": weapons_with_mobile_objects[i].position.x,
                                    "PositionY": weapons_with_mobile_objects[i].position.y,
                                    "PositionZ": weapons_with_mobile_objects[i].position.z,
                                    "OrientationW": weapons_with_mobile_objects[i].orientation.w,
                                    "OrientationX": weapons_with_mobile_objects[i].orientation.x,
                                    "OrientationY": weapons_with_mobile_objects[i].orientation.y,
                                    "OrientationZ": weapons_with_mobile_objects[i].orientation.z,
                                    "LocalDirectionX": weapons_with_mobile_objects[i].local_direction.x,
                                    "LocalDirectionY": weapons_with_mobile_objects[i].local_direction.y,
                                    "LocalDirectionZ": weapons_with_mobile_objects[i].local_direction.z,
                                    "MobilityType": weapons_with_mobile_objects[i].mobile_objects[j].mobility_type,
                                    "LocalPositionX": weapons_with_mobile_objects[i].mobile_objects[j].local_position.x,
                                    "LocalPositionY": weapons_with_mobile_objects[i].mobile_objects[j].local_position.y,
                                    "LocalPositionZ": weapons_with_mobile_objects[i].mobile_objects[j].local_position.z,
                                    "Fitness": 1})

        datasets[f"{o}_dataset_mobile_objects"] = dataset

        with open(f"{o}_dataset_mobile_objects.csv", mode='w') as file:
            fieldnames = ["Name",
                          "HullIndexParent",
                          "ScaleX",
                          "ScaleY",
                          "ScaleZ",
                          "PositionX",
                          "PositionY",
                          "PositionZ",
                          "OrientationW",
                          "OrientationX",
                          "OrientationY",
                          "OrientationZ",
                          "LocalDirectionX",
                          "LocalDirectionY",
                          "LocalDirectionZ",
                          "MobilityType",
                          "LocalPositionX",
                          "LocalPositionY",
                          "LocalPositionZ",
                          "Fitness"]

            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

            for row in dataset:
                # print(f"{row}\n")
                writer.writerow(row)
    
    ### Cannons ###
    cannons = xml.findall("WEAPONRYWEAPONSAI/Weapon/Cannons")
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon/Cannons': %d\n" % len(cannons))
    
    if len(cannons) > 0:
        cannons_number = list(map(int, [cannon.get("Number") for cannon in cannons]))
        print(f"CannonsNumber: {cannons_number}\n")

        firing_turn_maximums = list(map(int, [cannon.get("FiringTurnMaximum") for cannon in cannons]))
        firing_turn_maximums_ = [firing_turn_maximums[no] for no, rep in enumerate(cannons_number) for reps in range(rep)]
        print(f"FiringTurnMaximums: {firing_turn_maximums_}\n")
    
    ## WeaponCannons ##
    weapon_cannons = xml.findall("WEAPONRYWEAPONSAI/Weapon/Cannons/WeaponCannon")
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon/Cannons/WeaponCannon': %d\n" % len(weapon_cannons))
    
    if len(weapon_cannons) > 0:
        firing_turn_starts = list(map(int, values(weapon_cannons, nodes["WeaponCannonData"], "FiringTurnStart")))
        print(f"FiringTurnStarts: {firing_turn_starts}\n")

        firing_turn_ends = list(map(int, values(weapon_cannons, nodes["WeaponCannonData"], "FiringTurnEnd")))
        print(f"FiringTurnEnds: {firing_turn_ends}\n")

        rolls = list(map(float, values(weapon_cannons, nodes["WeaponCannonData"], "Roll")))
        print(f"Rolls: {rolls}\n")

        firing_positions_x = values(weapon_cannons, nodes["WeaponCannonData"], "FiringPositionX")
        firing_positions_y = values(weapon_cannons, nodes["WeaponCannonData"], "FiringPositionY")
        firing_positions_z = values(weapon_cannons, nodes["WeaponCannonData"], "FiringPositionZ")
        firing_positions = [Vector3(firing_positions_x[i], firing_positions_y[i], firing_positions_z[i]) for i in range(len(weapon_cannons))]
        print(f"FiringPositions: {firing_positions}\n")

        firing_directions_x = values(weapon_cannons, nodes["WeaponCannonData"], "FiringDirectionX")
        firing_directions_y = values(weapon_cannons, nodes["WeaponCannonData"], "FiringDirectionY")
        firing_directions_z = values(weapon_cannons, nodes["WeaponCannonData"], "FiringDirectionZ")
        firing_directions = [Vector3(firing_directions_x[i], firing_directions_y[i], firing_directions_z[i]) for i in range(len(weapon_cannons))]
        print(f"FiringDirections: {firing_directions}\n")

        cannons_ = [Cannon(firing_turn_maximums_[i], firing_turn_starts[i], firing_turn_ends[i], rolls[i], firing_positions[i], firing_directions[i]) for i in range(len(weapon_cannons))]
        print(f"{cannons_}\n")
    
    ### WeaponsWithCannons ###
    weapons = xml.findall("WEAPONRYWEAPONSAI/Weapon")
    weapons[:] = [weapon for weapon in weapons if weapon.find("Cannons") is not None]
    print(f"\nNo. of 'WEAPONRYWEAPONSAI/Weapon': %d\n" % len(weapons))
    
    if len(weapons) > 0:
        scales_x = values(weapons, nodes["ObjectData"], "ScaleX")
        scales_y = values(weapons, nodes["ObjectData"], "ScaleY")
        scales_z = values(weapons, nodes["ObjectData"], "ScaleZ")
        scales = [Vector3(scales_x[i], scales_y[i], scales_z[i]) for i in range(len(weapons))]
        print(f"Scales: {scales}\n")

        positions_x = values(weapons, nodes["ObjectData"], "PositionX")
        positions_y = values(weapons, nodes["ObjectData"], "PositionY")
        positions_z = values(weapons, nodes["ObjectData"], "PositionZ")
        positions = [Vector3(positions_x[i], positions_y[i], positions_z[i]) for i in range(len(weapons))]
        print(f"Positions: {positions}\n")

        orientations_w = values(weapons, nodes["ObjectData"], "OrientationW")
        orientations_x = values(weapons, nodes["ObjectData"], "OrientationX")
        orientations_y = values(weapons, nodes["ObjectData"], "OrientationY")
        orientations_z = values(weapons, nodes["ObjectData"], "OrientationZ")
        orientations = [Quaternion(orientations_w[i], orientations_x[i], orientations_y[i], orientations_z[i]) for i in range(len(weapons))]
        print(f"Orientations: {orientations}\n")

        hull_index_parents = list(map(int, values(weapons, nodes["ComponentData"], "HullIndexParent")))
        print(f"HullIndexParents: {hull_index_parents}\n")

        local_directions_x = values(weapons, nodes["ComponentData"], "LocalDirectionX")
        local_directions_y = values(weapons, nodes["ComponentData"], "LocalDirectionY")
        local_directions_z = values(weapons, nodes["ComponentData"], "LocalDirectionZ")
        local_directions = [Vector3(local_directions_x[i], local_directions_y[i], local_directions_z[i]) for i in range(len(weapons))]
        print(f"LocalDirections: {local_directions}\n")

        k, weapons_with_cannons = 0, []
        for i in range(len(cannons_number)):
            cannons__ = []
            for j in range(cannons_number[i]):
                cannons__ += [cannons_[k]]
                k += 1
            weapons_with_cannons += [WeaponWithCannons(scales[i], positions[i], orientations[i], hull_index_parents[i], local_directions[i], cannons__)]

        dataset = []
        for i in range(len(weapons_with_cannons)):
            for j in range(len(weapons_with_cannons[i].cannons)):
                dataset.append({"Name": f"{o}-baseline",
                                "HullIndexParent": weapons_with_cannons[i].hull_index_parent,
                                "ScaleX": weapons_with_cannons[i].scale.x,
                                "ScaleY": weapons_with_cannons[i].scale.y,
                                "ScaleZ": weapons_with_cannons[i].scale.z,
                                "PositionX": weapons_with_cannons[i].position.x,
                                "PositionY": weapons_with_cannons[i].position.y,
                                "PositionZ": weapons_with_cannons[i].position.z,
                                "OrientationW": weapons_with_cannons[i].orientation.w,
                                "OrientationX": weapons_with_cannons[i].orientation.x,
                                "OrientationY": weapons_with_cannons[i].orientation.y,
                                "OrientationZ": weapons_with_cannons[i].orientation.z,
                                "LocalDirectionX": weapons_with_cannons[i].local_direction.x,
                                "LocalDirectionY": weapons_with_cannons[i].local_direction.y,
                                "LocalDirectionZ": weapons_with_cannons[i].local_direction.z,
                                "FiringTurnMaximum": weapons_with_cannons[i].cannons[j].firing_turn_maximum,
                                "FiringTurnStart": weapons_with_cannons[i].cannons[j].firing_turn_start,
                                "FiringTurnEnd": weapons_with_cannons[i].cannons[j].firing_turn_end,
                                "Roll": weapons_with_cannons[i].cannons[j].roll,
                                "FiringPositionX": weapons_with_cannons[i].cannons[j].firing_position.x,
                                "FiringPositionY": weapons_with_cannons[i].cannons[j].firing_position.y,
                                "FiringPositionZ": weapons_with_cannons[i].cannons[j].firing_position.z,
                                "FiringDirectionX": weapons_with_cannons[i].cannons[j].firing_direction.x,
                                "FiringDirectionY": weapons_with_cannons[i].cannons[j].firing_direction.y,
                                "FiringDirectionZ": weapons_with_cannons[i].cannons[j].firing_direction.z,
                                "Fitness": 1})
                
        datasets[f"{o}_dataset_cannons"] = dataset

        with open(f"{o}_dataset_cannons.csv", mode='w') as file:
            fieldnames = ["Name",
                          "HullIndexParent",
                          "ScaleX",
                          "ScaleY",
                          "ScaleZ",
                          "PositionX",
                          "PositionY",
                          "PositionZ",
                          "OrientationW",
                          "OrientationX",
                          "OrientationY",
                          "OrientationZ",
                          "LocalDirectionX",
                          "LocalDirectionY",
                          "LocalDirectionZ",
                          "FiringTurnMaximum",
                          "FiringTurnStart",
                          "FiringTurnEnd",
                          "Roll",
                          "FiringPositionX",
                          "FiringPositionY",
                          "FiringPositionZ",
                          "FiringDirectionX",
                          "FiringDirectionY",
                          "FiringDirectionZ",
                          "Fitness"]

            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()

            for row in dataset:
                # print(f"{row}\n")
                writer.writerow(row)

###### Working with Argos.xml #######

No. of 'WEAPONRYWEAPONSAI/Weapon/MobileObjects': 0


No. of 'WEAPONRYWEAPONSAI/Weapon': 0


No. of 'WEAPONRYWEAPONSAI/Weapon/Cannons': 19

CannonsNumber: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

FiringTurnMaximums: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


No. of 'WEAPONRYWEAPONSAI/Weapon/Cannons/WeaponCannon': 19

FiringTurnStarts: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

FiringTurnEnds: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Rolls: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

FiringPositions: [Vector3(0.0, -0.467, -1.049), Vector3(0.0, -0.467, -1.049), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.0, 0.12), Vector3(0.0, 0.6, 0.0), 

## 1.2. MovementAI

In [7]:
for o in objects:
    xml = ET.parse(f"{o}.xml").getroot()
    movement_ai = xml.findall("MOVEMENTAI")
    
    movement_ai_ = {}
    for elem in movement_ai:
        for node in elem.iter():
            movement_ai_.update(node.attrib)
    print(movement_ai_)

    dataset = [{"Name": f"{o}-baseline", **movement_ai_}]
    dataset[0]["Fitness"] = 1
    
    datasets[f"{o}_dataset_movement_ai"] = dataset
    
    with open(f"{o}_dataset_movement_ai.csv", mode='w') as file:
        fieldnames = ["Name"] + list(movement_ai_.keys()) + ["Fitness"]

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for row in dataset:
            # print(f"{row}\n")
            writer.writerow(row)

{'AIModuleMovementType': '6', 'Acceleration': '30', 'MinimumSpeed': '0', 'MaximumSpeed': '1000', 'ReachDestinationAtFullLinearSpeed': '0', 'AccelerationAngular': '1', 'MaximumSpeedAngular': '0.5', 'ReachDestinationAtFullAngularSpeed': '1'}
{'AIModuleMovementType': '6', 'Acceleration': '30', 'MinimumSpeed': '0', 'MaximumSpeed': '1000', 'ReachDestinationAtFullLinearSpeed': '0', 'AccelerationAngular': '3', 'MaximumSpeedAngular': '3', 'ReachDestinationAtFullAngularSpeed': '0'}
{'AIModuleMovementType': '6', 'Acceleration': '300', 'MinimumSpeed': '30', 'MaximumSpeed': '1000', 'ReachDestinationAtFullLinearSpeed': '0', 'AccelerationAngular': '1', 'MaximumSpeedAngular': '10', 'ReachDestinationAtFullAngularSpeed': '0'}
{'AIModuleMovementType': '4', 'Acceleration': '300', 'MinimumSpeed': '50', 'MaximumSpeed': '100', 'ReachDestinationAtFullLinearSpeed': '0', 'AccelerationAngular': '5', 'MaximumSpeedAngular': '5', 'ReachDestinationAtFullAngularSpeed': '0'}


## 1.3. AIUnit (BehaviourUnitAI)

In [8]:
for o in objects:
    xml = ET.parse(f"{o}.xml").getroot()
    ai_unit = xml.findall("AIUNIT")
    
    unit_value = list(map(int, [elem.get("UnitValue") for elem in ai_unit]))[0]
    print(f"UnitValue: {unit_value}\n")
    
    behaviour_unit_ai_data = xml.findall("AIUNIT/BehaviourUnitAIData")
    print(f"\nNo. of 'AIUNIT/BehaviourUnitAIData': %d\n" % len(behaviour_unit_ai_data))
    
    behaviour_unit_ai_data_ = [elem.attrib for elem in behaviour_unit_ai_data]
    print(behaviour_unit_ai_data_)
    
    dataset = []
    for i in range(len(behaviour_unit_ai_data_)):
        dataset += [{"Name": f"{o}-baseline", "UnitValue": unit_value, **behaviour_unit_ai_data_[i]}]
        dataset[i]["Fitness"] = 1
        
    datasets[f"{o}_dataset_ai_unit"] = dataset
    
    with open(f"{o}_dataset_ai_unit.csv", mode='w') as file:
        fieldnames = ["Name", "UnitValue"] + list(behaviour_unit_ai_data_[0].keys()) + ["Fitness"]

        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for row in dataset:
            # print(f"{row}\n")
            writer.writerow(row)

UnitValue: -1


No. of 'AIUNIT/BehaviourUnitAIData': 3

[{'AIFormationType': '2', 'AIFormationSeparationMultiplier': '5', 'AIGroupBehaviourType': '-1', 'AIGroupBehaviourDistance': '0', 'AIGroupAttackBehaviourType': '-1', 'AIGroupAttackSpeed': '0', 'AIBehaviourType': '4', 'AIBehaviourDistance': '200', 'AIAttackBehaviourType': '0', 'AIAttackSpeed': '100'}, {'AIFormationType': '2', 'AIFormationSeparationMultiplier': '5', 'AIGroupBehaviourType': '-1', 'AIGroupBehaviourDistance': '0', 'AIGroupAttackBehaviourType': '-1', 'AIGroupAttackSpeed': '0', 'AIBehaviourType': '4', 'AIBehaviourDistance': '100', 'AIAttackBehaviourType': '0', 'AIAttackSpeed': '50'}, {'AIFormationType': '2', 'AIFormationSeparationMultiplier': '5', 'AIGroupBehaviourType': '-1', 'AIGroupBehaviourDistance': '0', 'AIGroupAttackBehaviourType': '-1', 'AIGroupAttackSpeed': '0', 'AIBehaviourType': '4', 'AIBehaviourDistance': '200', 'AIAttackBehaviourType': '0', 'AIAttackSpeed': '100'}]
UnitValue: -1


No. of 'AIUNIT/BehaviourUnit

# 2. Mutant creation for training

In [9]:
import random
import numpy as np

In [10]:
''' Subjects a given value to a mutation process.

The result of the mutation can be: (0) value incremented by a percent,
(1) value decremented by a percent, or (2) default value.

:param value: input value
:type value: float
:param percentage: mutation's percent
:type percentage: int
:returns: mutated value
:rtype: float
'''
def mutate_value(value: float, percentage: int) -> float:
    mutation = random.randrange(0, 2 + 1, 1)  # value range: (0, 1, 2)

    if mutation == 0:  # incremental mutation
        return float(value + value * (percentage / 100))
    elif mutation == 1:  # decremental mutation
        return float(value - value * (percentage / 100))
    else:
        return value


''' Subjects a given set of properties belonging to an specific dataset to a mutation process.

Usage:
  1. mutate_props(dataset_, props=props, percentage=[10, 15])
  2. mutate_props(dataset_, props=props, percentage=14)

:param dataset: input dataset
:type dataset: dictionary array
:param props: set of properties
:type props: str array
:param percentage: mutation's percent
:type percentage: int or list
:returns: mutated dataset
:rtype: dictionary array
'''
def mutate_props(dataset, props, percentage, index=0):
    if not isinstance(percentage, (int, list)):
        return

    dataset_mut = []

    isrange = True if isinstance(percentage, list) else False
    percentage_ = random.randrange(
        percentage[0], percentage[1] + 1, 1) if isinstance(percentage, list) else percentage

    for row in dataset:
        data = row.copy()

        if isrange:
            percentage_ = random.randrange(percentage[0], percentage[1] + 1, 1)

        for prop in props:
            data[prop] = mutate_value(float(row[prop]), percentage_)

        data["Name"] = data["Name"].replace("baseline", "mutated")
        data["Fitness"] = 1 - percentage_ / 100

        dataset_mut.append(data)

        index += 1

    return dataset_mut


''' Subjects a given model to a mutation process.

:param dataset: input dataset
:type dataset: dictionary array
:param props: set of properties
:type props: str array
:param percentage: mutation's percent
:type percentage: int or list
:param times: number of times the mutation is performed (default: 1)
:type times: int
:param filename: name of the csv file where to export the dataset (default: None)
:type filename: str
'''
def mutate_model(dataset, props, percentage, times=1, filename=None, fieldnames=["Name", "Fitness"]):
    dataset_all = np.array(dataset[:])

    for t in range(times):
        dataset_mut = np.array(mutate_props(
            dataset[:], props, percentage))
        dataset_all = np.concatenate((dataset_all, dataset_mut))

    if filename is not None:
        with open(filename + ".csv", mode='w') as file:
            writer = csv.DictWriter(file, fieldnames)
            writer.writeheader()

            for row in dataset_all:
                writer.writerow(row)


for key in datasets.keys():
    if "mobile_objects" in key:
        fieldnames = ["Name", "HullIndexParent",
                      "ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "MobilityType", "LocalPositionX", "LocalPositionY", "LocalPositionZ",
                      "Fitness"]
        mutate_model(datasets[key], props=["LocalDirectionX", "LocalDirectionY", "LocalDirectionZ", "LocalPositionX", "LocalPositionY", "LocalPositionZ"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "cannons" in key:
        fieldnames = ["Name", "HullIndexParent",
                      "ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll",
                      "FiringPositionX","FiringPositionY","FiringPositionZ",
                      "FiringDirectionX","FiringDirectionY","FiringDirectionZ",
                      "Fitness"]
        mutate_model(datasets[key], props=["LocalDirectionX", "LocalDirectionY", "LocalDirectionZ", "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll", "FiringPositionX", "FiringPositionY", "FiringPositionZ", "FiringDirectionX", "FiringDirectionY", "FiringDirectionZ"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "movement_ai" in key:
        fieldnames = ["Name"] + list(movement_ai_.keys()) + ["Fitness"]
        mutate_model(datasets[key], props=["Acceleration", "MinimumSpeed", "MaximumSpeed", "ReachDestinationAtFullLinearSpeed", "AccelerationAngular", "MaximumSpeedAngular", "ReachDestinationAtFullAngularSpeed"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)
    elif "ai_unit" in key:
        fieldnames = ["Name", "UnitValue"] + list(behaviour_unit_ai_data_[0].keys()) + ["Fitness"]
        mutate_model(datasets[key], props=["AIBehaviourDistance", "AIAttackSpeed"], percentage=[10, 15], times=49, filename=f"{key}_mut", fieldnames=fieldnames)


# 3. Model classification (10-fold Cross-validation)

In [11]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

import seaborn as sb
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [12]:
for key in datasets.keys():
    if "mobile_objects" in key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "MobilityType", "LocalPositionX", "LocalPositionY", "LocalPositionZ"]
    elif "cannons" in key:
        fieldnames = ["ScaleX", "ScaleY", "ScaleZ",
                      "PositionX", "PositionY", "PositionZ",
                      "OrientationW", "OrientationX", "OrientationY", "OrientationZ",
                      "LocalDirectionX", "LocalDirectionY", "LocalDirectionZ",
                      "FiringTurnMaximum", "FiringTurnStart", "FiringTurnEnd", "Roll",
                      "FiringPositionX","FiringPositionY","FiringPositionZ",
                      "FiringDirectionX","FiringDirectionY","FiringDirectionZ"]
    elif "movement_ai" in key:
        fieldnames = list(movement_ai_.keys())
    elif "ai_unit" in key:
        fieldnames = ["UnitValue"] + list(behaviour_unit_ai_data_[0].keys())
    
    print(f"### {key}_mut.csv ###\n")
    model_data = pd.read_csv(f"{key}_mut.csv")

    all_inputs = model_data[fieldnames].values
    all_classes = model_data["Name"].values
    
    print(all_inputs[:5], end='\n\n')
    print(f"Classes: {all_classes}", end='\n\n')
    
    (training_inputs,
     testing_inputs,
     training_classes,
     testing_classes) = train_test_split(all_inputs, all_classes, train_size=0.80, random_state=456)

    print(f"Training set: {training_inputs.shape}")
    print(f"Testing set: {testing_inputs.shape}", end='\n\n')
    
    ### DecisionTreeClassifier ###
    # Create the classifier.
    decision_tree_classifier = DecisionTreeClassifier()
    # Train the classifier on the training set.
    decision_tree_classifier.fit(training_inputs, training_classes)
    # Validate the classifier on the testing set using classification accuracy.
    print("DecisionTreeClassifier")
    print(decision_tree_classifier.score(testing_inputs, testing_classes))
    print(decision_tree_classifier.predict(testing_inputs[:1, :]), end='\n\n')

    ### Model accuracies ###
    plt.title("Model accuracies")
    # DecisionTreeClassifier
    model_accuracies = []
    for repetition in range(1000):
        (training_inputs,
         testing_inputs,
         training_classes,
         testing_classes) = train_test_split(all_inputs, all_classes, train_size=0.75)

        decision_tree_classifier = DecisionTreeClassifier()
        decision_tree_classifier.fit(training_inputs, training_classes)
        
        classifier_accuracy = decision_tree_classifier.score(testing_inputs, testing_classes)
        model_accuracies.append(classifier_accuracy)
    sb.distplot(model_accuracies, label="DecisionTreeClassifier")
    plt.legend()
    plt.savefig(f"plots/{key}_model_accuracies.pdf")
    plt.close("all")
    
    ### 10-fold Cross-validation ###
    # DecisionTreeClassifier(max_depth=4)
    decision_tree_classifier = DecisionTreeClassifier()

    # cross_val_score returns a list of the scores, which we can visualize
    # to get a reasonable estimate of the classifier's performance
    cv_scores = cross_val_score(decision_tree_classifier, all_inputs, all_classes, cv=10)
    sb.distplot(cv_scores)
    plt.title(f"Average score: {np.mean(cv_scores)}")
    plt.savefig(f"plots/{key}_model_cv_scores.pdf")
    plt.close("all")

### Argos_dataset_cannons_mut.csv ###

[[ 1.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     1.     0.     0.     0.     0.     0.    -0.467 -1.049
   0.     0.    -1.   ]
 [ 1.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     1.     0.     0.     0.     0.     0.    -0.467 -1.049
   0.     0.    -1.   ]
 [ 1.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     1.     0.     0.     0.     0.     0.     0.     0.12
   0.     0.     1.   ]
 [ 1.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     1.     0.     0.     0.     0.     0.     0.     0.12
   0.     0.     1.   ]
 [ 1.     1.     1.     0.     0.     0.     1.     0.     0.     0.
   0.     0.     1.     0.     0.     0.     0.     0.     0.     0.12
   0.     0.     1.   ]]

Classes: ['Argos-baseline' 'Argos-baseline' 'Argos-baseline' 'Argos-baseline'
 'Argos-baseline' 'Argos-baseline' 'Argos-baseline' 'Argos-

### Orion_dataset_mobile_objects_mut.csv ###

[[0.22 0.22 0.22 0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   3.
  0.   0.   0.  ]
 [0.22 0.22 0.22 0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   3.
  0.   0.   0.  ]
 [1.   1.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   3.
  0.   0.   0.  ]
 [1.   1.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   3.
  0.   0.   0.  ]
 [1.   1.   1.   0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   3.
  0.   0.   0.  ]]

Classes: ['Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'O

### Orion_dataset_cannons_mut.csv ###

[[ 0.22  0.22  0.22  0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.   17.   15.   17.    0.    0.    0.    0.3   0.    0.    1.  ]
 [ 0.22  0.22  0.22  0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.   17.   15.   17.    0.    0.    0.    0.3   0.    0.    1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.   17.   15.   17.    0.    0.    0.    0.3   0.    0.    1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.   17.   15.   17.    0.    0.    0.    0.3   0.    0.    1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.   17.   15.   17.    0.    0.    0.    0.3   0.    0.    1.  ]]

Classes: ['Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-mutated' 'Ori

### Teuthus_dataset_mobile_objects_mut.csv ###

[[ 1.      1.      1.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      1.      3.      0.      0.      0.    ]
 [ 1.      1.      1.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      1.      3.      0.      0.      0.    ]
 [ 1.      1.      1.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      1.      3.      0.      0.      0.    ]
 [ 1.      1.      1.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      1.      3.      0.      0.      0.    ]
 [ 1.      1.      1.      0.      0.      0.      1.      0.      0.
   0.      0.      0.      1.     11.      0.      0.      0.1575]]

Classes: ['Teuthus-baseline' 'Teuthus-baseline' 'Teuthus-baseline'
 'Teuthus-baseline' 'Teuthus-baseline' 'Teuthus-baseline'
 'Teuthus-baseline' 'Teuthus-baseline' 'Teuthus-baseline'
 'Teuthus-baseline' 'Teuthus-mutated' 'Teuthus-mutated' 'Teuthus-mutated'
 'Teuthu

### Teuthus_dataset_cannons_mut.csv ###

[[ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.    0.    0.    0.1   0.    0.    1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.   -0.11  0.11  0.1  -0.01  0.01  1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.    0.11  0.11  0.1   0.01  0.01  1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.   -0.11 -0.11  0.1  -0.01 -0.01  1.  ]
 [ 1.    1.    1.    0.    0.    0.    1.    0.    0.    0.    0.    0.
   1.    0.    0.    0.    0.    0.11 -0.11  0.1   0.01 -0.01  1.  ]]

Classes: ['Teuthus-baseline' 'Teuthus-baseline' 'Teuthus-baseline' ...
 'Teuthus-mutated' 'Teuthus-mutated' 'Teuthus-mutated']

Training set: (920, 23)
Testing set: (230, 23)

DecisionTreeClassifier
0.9608695652173913
['Teuthus-mutated']

### Vermis_da

### Argos_dataset_movement_ai_mut.csv ###

[[6.00e+00 3.00e+01 0.00e+00 1.00e+03 0.00e+00 1.00e+00 5.00e-01 1.00e+00]
 [6.00e+00 2.61e+01 0.00e+00 8.70e+02 0.00e+00 8.70e-01 4.35e-01 1.13e+00]
 [6.00e+00 3.00e+01 0.00e+00 1.00e+03 0.00e+00 8.60e-01 5.70e-01 1.14e+00]
 [6.00e+00 2.55e+01 0.00e+00 8.50e+02 0.00e+00 1.15e+00 5.00e-01 8.50e-01]
 [6.00e+00 2.64e+01 0.00e+00 1.00e+03 0.00e+00 1.00e+00 5.60e-01 1.12e+00]]

Classes: ['Argos-baseline' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' 'Argos-mutated'
 'Argos-mutated' 'Argos-mutated' 'Argos-mutated' '

### Orion_dataset_ai_unit_mut.csv ###

[[-1.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00  4.e+00  3.e+02
   0.e+00  1.e+02]
 [-1.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00  4.e+00  2.e+02
   0.e+00  1.e+02]
 [-1.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00  9.e+00  1.e+05
   0.e+00  1.e+03]
 [-1.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00  9.e+00  1.e+05
   0.e+00  1.e+03]
 [-1.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00 -1.e+00  0.e+00  9.e+00  1.e+05
   0.e+00  1.e+03]]

Classes: ['Orion-baseline' 'Orion-baseline' 'Orion-baseline' 'Orion-baseline'
 'Orion-baseline' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-mutated' 'Orion-mutated' 'Orion-mutated'
 'Orion-mutated' 'Orion-m